# QQP

In [7]:
#Contexto SparkSQL
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()

In [8]:
sqlContext = SQLContext(sc)

In [86]:
from pyspark.sql.types import IntegerType

In [15]:
QQP = sqlContext.read.csv('all_data.csv', header = 'true')

In [23]:
#Número de Categorias y nombre de ellas
len(QQP.columns), QQP.columns

(15,
 ['producto',
  'presentacion',
  'marca',
  'categoria',
  'catalogo',
  'precio',
  'fechaRegistro',
  'cadenaComercial',
  'giro',
  'nombreComercial',
  'direccion',
  'estado',
  'municipio',
  'latitud',
  'longitud'])

In [20]:
#Echando un vistazo a la estructura de las variables
QQP.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [21]:
#Contar el número de observaciones o registros
QQP.count()

62530715

In [24]:
#Valores diferentes para la variable cadenaComercial (cadenas comerciales monitoreadas)
QQP.select('cadenaComercial').distinct(), QQP.select('cadenaComercial').distinct().count()

(DataFrame[cadenaComercial: string], 706)

In [25]:
cadenasComerciales = QQP.select('cadenaComercial').distinct() #crea un DF con los valores distintos de cadenaComercial

In [28]:
QQP.select('cadenaComercial').distinct().show() #muestra los diferentes valores de la categoria cadenaComercial.
#poco util ya que no muestra una lista completa y no permite notar si los nombres de las diferentes cadenasComerciales 
#estan escritos consistentemente


+--------------------+
|     cadenaComercial|
+--------------------+
|JULIO CEPEDA JUGU...|
|           WOOLWORTH|
| MARISCOS LA SEPTIMA|
|PAPELERIA EL TINTERO|
|LIBRERIA DE SANCH...|
|PAPELERIA Y LIBRE...|
|     PLAZA DEL LIBRO|
|MERCADO ALONSO FE...|
|     MERCADO LA CRUZ|
|     ZAPATERIA PAKAR|
|              SIGUBA|
|           MEGA TOYS|
| MARISCOS VILLA AZUL|
|    BODEGAS VICTORIA|
|     MINISUPER INALA|
|              S MART|
|    LIBRERIA ATHENAS|
|        FARMACIAS M+|
| MUEBLERIA CREDILAND|
|   LOYEP (UNIFORMES)|
+--------------------+
only showing top 20 rows



## Productos monitoreados por entidad

In [36]:
step1 = QQP.groupBy("estado", "producto").count() #agrupamos por entidad y producto

In [39]:
step1.show()

+--------------------+--------------------+-----+
|              estado|            producto|count|
+--------------------+--------------------+-----+
|              MÉXICO|TINTE PARA EL CAB...|44007|
|              MÉXICO|         TELEVISORES|29702|
|              MÉXICO|              ACELGA| 7691|
|              MÉXICO|       QUESO. COTIJA| 4414|
|    DISTRITO FEDERAL|              AZUCAR|18078|
|              MÉXICO|      DESENFRIOL-ITO|  642|
|             JALISCO|               ARROZ|11735|
|              OAXACA|PEDIALYTE. ELECTR...|  302|
|            TLAXCALA|        AGUA SIN GAS|14505|
|VERACRUZ DE IGNAC...|              TOMATE|  652|
| MICHOACÁN DE OCAMPO|         PAN DE CAJA|13003|
|             YUCATÁN|       FLAGENASE 400|  313|
| MICHOACÁN DE OCAMPO|              ECTIVA|   39|
|             YUCATÁN|        SALSA CATSUP| 6549|
|             YUCATÁN|            CLAVULIN|  183|
|             YUCATÁN|            CAPOTENA|  271|
|             JALISCO|       FLAGENASE 400|  699|


In [40]:
type(step1)

pyspark.sql.dataframe.DataFrame

In [41]:
step1.filter(step1['estado'] == 'MÉXICO').show()


+------+--------------------+-----+
|estado|            producto|count|
+------+--------------------+-----+
|MÉXICO|TINTE PARA EL CAB...|44007|
|MÉXICO|         TELEVISORES|29702|
|MÉXICO|              ACELGA| 7691|
|MÉXICO|       QUESO. COTIJA| 4414|
|MÉXICO|      DESENFRIOL-ITO|  642|
|MÉXICO|PIERNA DE CERDO E...|   78|
|MÉXICO|                HABA| 4796|
|MÉXICO|            SENSIBIT|  516|
|MÉXICO|       DAFLON 500 M.| 4244|
|MÉXICO|           SINGULAIR| 8669|
|MÉXICO|DETERGENTE P/TRASTES|50337|
|MÉXICO|      LAPIZ ADHESIVO|20589|
|MÉXICO|        PAPEL LUSTRE| 9362|
|MÉXICO|   JUGUETES ARMABLES| 6328|
|MÉXICO|        ALKA-SELTZER| 3139|
|MÉXICO|           NISTATINA|  359|
|MÉXICO|         KETOCONAZOL| 2850|
|MÉXICO|             PENAMOX|  379|
|MÉXICO|     JUGUETES VARIOS|  617|
|MÉXICO|     AUGMENTIN 12 H.| 4143|
+------+--------------------+-----+
only showing top 20 rows



In [78]:
step1.printSchema()

root
 |-- estado: string (nullable = true)
 |-- producto: string (nullable = true)
 |-- count: long (nullable = false)



In [87]:
#cambiar el tipo de dato en columna count
step1 = step1.withColumn("count", step1["count"].cast(IntegerType()))

In [89]:
#verificamos el cambio
step1.printSchema()

root
 |-- estado: string (nullable = true)
 |-- producto: string (nullable = true)
 |-- count: integer (nullable = false)



In [48]:
estados_lista = step1.select('estado').distinct() #recuperamos los posibles valores que puede tomar la variable estado

In [46]:
estados_lista.show()

+--------------------+
|              estado|
+--------------------+
|        QUINTANA ROO|
|          NUEVO LEÓN|
|             SINALOA|
|             TABASCO|
|     BAJA CALIFORNIA|
|            TLAXCALA|
|COAHUILA DE ZARAGOZA|
|                null|
|       ESQ. SUR 125"|
| COL. EDUARDO GUERRA|
|             CHIAPAS|
|VERACRUZ DE IGNAC...|
|              SONORA|
|             YUCATÁN|
| MICHOACÁN DE OCAMPO|
|             DURANGO|
|            GUERRERO|
|             NAYARIT|
|           CHIHUAHUA|
|    DISTRITO FEDERAL|
+--------------------+
only showing top 20 rows



In [60]:
#creamos una lista con todos los nombres de los estados, luego notamos que hay entradas que no son estados. 
lista = [str(row.estado) for row in estados_lista.collect()]

In [122]:
lista[7]

'None'

In [128]:
lista

['QUINTANA ROO',
 'NUEVO LEÓN',
 'SINALOA',
 'TABASCO',
 'BAJA CALIFORNIA',
 'TLAXCALA',
 'COAHUILA DE ZARAGOZA',
 'None',
 ' ESQ. SUR 125"',
 ' COL. EDUARDO GUERRA',
 'CHIAPAS',
 'VERACRUZ DE IGNACIO DE LA LLAVE',
 'SONORA',
 'YUCATÁN',
 'MICHOACÁN DE OCAMPO',
 'DURANGO',
 'GUERRERO',
 'NAYARIT',
 'CHIHUAHUA',
 'DISTRITO FEDERAL',
 'estado',
 'HIDALGO',
 'ZACATECAS',
 'GUANAJUATO',
 'TAMAULIPAS',
 'SAN LUIS POTOSÍ',
 'MORELOS',
 'AGUASCALIENTES',
 'OAXACA',
 'PUEBLA',
 'BAJA CALIFORNIA SUR',
 'JALISCO',
 'CAMPECHE',
 'QUERÉTARO',
 'COLIMA',
 'MÉXICO']

In [130]:
#removemos los valores invalidos para el campo estado
lista.remove('None') 
lista.remove(' ESQ. SUR 125"')
lista.remove(' COL. EDUARDO GUERRA')
lista.remove('estado')

In [132]:
len(lista) #ya tenemos solo los 32 estados

32

In [133]:
C = []
for i in range(len(lista)):
    tmp = step1.where(step1.estado == lista[i]) #filtrar por cada nombre de estado que aparece en la lista
    tmp = tmp.orderBy("count", ascending = False) #ordenar descendentemente el filtrado anterior
    producto = tmp.head(1)[0] #puede pasar que sea vacia
    info = (lista[i],producto[1],producto[2]) #creamos una tupla en la que tenga la inforamcion del estado, producto y cantidad de producto
    C.append(info)

In [134]:
#C contiene la informacion de los productos mas monitoreados por entidad y 
#el conteo de los registros que se tiene de ellos
C

[('QUINTANA ROO', 'FUD', 34846),
 ('NUEVO LEÓN', 'DETERGENTE P/ROPA', 50307),
 ('SINALOA', 'REFRESCO', 33115),
 ('TABASCO', 'REFRESCO', 28754),
 ('BAJA CALIFORNIA', 'REFRESCO', 37243),
 ('TLAXCALA', 'REFRESCO', 43904),
 ('COAHUILA DE ZARAGOZA', 'FUD', 28613),
 ('CHIAPAS', 'REFRESCO', 14452),
 ('VERACRUZ DE IGNACIO DE LA LLAVE', 'REFRESCO', 14576),
 ('SONORA', 'REFRESCO', 34731),
 ('YUCATÁN', 'LECHE ULTRAPASTEURIZADA', 35991),
 ('MICHOACÁN DE OCAMPO', 'DETERGENTE P/ROPA', 40144),
 ('DURANGO', 'DETERGENTE P/ROPA', 9696),
 ('GUERRERO', 'REFRESCO', 8932),
 ('NAYARIT', 'REFRESCO', 8003),
 ('CHIHUAHUA', 'DETERGENTE P/ROPA', 15982),
 ('DISTRITO FEDERAL', 'REFRESCO', 287463),
 ('HIDALGO', 'REFRESCO', 18137),
 ('ZACATECAS', 'DETERGENTE P/ROPA', 20884),
 ('GUANAJUATO', 'REFRESCO', 49441),
 ('TAMAULIPAS', 'REFRESCO', 22457),
 ('SAN LUIS POTOSÍ', 'FUD', 10164),
 ('MORELOS', 'REFRESCO', 9632),
 ('AGUASCALIENTES', 'FUD', 12005),
 ('OAXACA', 'LECHE ULTRAPASTEURIZADA', 18078),
 ('PUEBLA', 'REFRESCO', 

# Variedad de productos por cadena comercial

In [135]:
#variedad de productos por cadena comercial
cadenas_comerciales = QQP.groupBy("cadenaComercial", "producto").count()

In [136]:
cadenas_comerciales.head(2)

[Row(cadenaComercial='I.S.S.S.T.E.', producto='MOLE ROJO EN PASTA', count=6090),
 Row(cadenaComercial='WAL-MART', producto='PAN DE CAJA', count=66623)]

In [142]:
cadenas_comerciales.printSchema()

root
 |-- cadenaComercial: string (nullable = true)
 |-- producto: string (nullable = true)
 |-- count: long (nullable = false)



In [138]:
#conteos de la tabla anterior para conocer cuantos productos se monitorean por cadena comercial
cadenas_comerciales_2 = cadenas_comerciales.groupBy('CadenaComercial').count() 

In [139]:
cadenas_comerciales_2.head(2)

[Row(CadenaComercial='WOOLWORTH', count=95),
 Row(CadenaComercial='LIBRERIA DE SANCHO PANZA', count=31)]

In [149]:
#Ordenamos las cadenas de forma descendente y así encontraremos las que tienen más productos monitoreados en la 
#primel fila de nuestro DataFrame
cadenas_comerciales_2 = cadenas_comerciales_2.orderBy("count", ascending=False)

In [150]:
#Las 5 cadenas comerciales con más productos monitoreados
cadenas_comerciales_2.head(5)

[Row(CadenaComercial='SORIANA', count=1059),
 Row(CadenaComercial='WAL-MART', count=1051),
 Row(CadenaComercial='MEGA COMERCIAL MEXICANA', count=1049),
 Row(CadenaComercial='COMERCIAL MEXICANA', count=1036),
 Row(CadenaComercial='CHEDRAUI', count=1026)]

# Canasta básica

In [153]:
QQP.head(2)

[Row(producto='CUADERNO FORMA ITALIANA', presentacion='96 HOJAS PASTA DURA. CUADRICULA CHICA', marca='ESTRELLA', categoria='MATERIAL ESCOLAR', catalogo='UTILES ESCOLARES', precio='25.9', fechaRegistro='2011-05-18 00:00:00.000', cadenaComercial='ABASTECEDORA LUMEN', giro='PAPELERIAS', nombreComercial='ABASTECEDORA LUMEN SUCURSAL VILLA COAPA', direccion='CANNES No. 6 ESQ. CANAL DE MIRAMONTES', estado='DISTRITO FEDERAL', municipio='TLALPAN                                 ', latitud='19.29699', longitud='-99.125417'),
 Row(producto='CRAYONES', presentacion='CAJA 12 CERAS. JUMBO. C.B. 201423', marca='CRAYOLA', categoria='MATERIAL ESCOLAR', catalogo='UTILES ESCOLARES', precio='27.5', fechaRegistro='2011-05-18 00:00:00.000', cadenaComercial='ABASTECEDORA LUMEN', giro='PAPELERIAS', nombreComercial='ABASTECEDORA LUMEN SUCURSAL VILLA COAPA', direccion='CANNES No. 6 ESQ. CANAL DE MIRAMONTES', estado='DISTRITO FEDERAL', municipio='TLALPAN                                 ', latitud='19.29699', long

In [156]:
QQP.select('catalogo').distinct().show()

+--------------------+
|            catalogo|
+--------------------+
|             BASICOS|
|            MERCADOS|
|                null|
|    UTILES ESCOLARES|
|   ELECTRODOMESTICOS|
|            catalogo|
|           NAVIDEÑOS|
|REGIONALES MONTERREY|
|            JUGUETES|
|               TENIS|
|        MEDICAMENTOS|
| PESCADOS Y MARISCOS|
|  FRUTAS Y LEGUMBRES|
+--------------------+



###### Para la creación de la canasta básica iba a considerar  los prouctos contenidos en el catalogo BASICOS. Para hacer la comparativa se puede realizar un dashboard que permita seleccionar algún producto de la canasta básica y una fecha, que incluya un mapa donde indique el precio de dicho producto en cada estado (una opcion es estimar el precio del producto por estado calculando un promedio de todos los precios que caen en el). También se puede colorear el precio para indicar si es alto o bajo y de esta manera poder identificar las ciudades más caras. Dado que no estoy por completo familiarizada con SparkSQL me estaba resultando muy lento el proceso de la creación de este dashboard. 